In [4]:
import re
import numpy as np
import pandas as pd

import torch
from torch import nn
import torch.nn.functional as F

import sys
sys.path.append('src/')
from smiles_lstm.model.smiles_vocabulary import SMILESTokenizer, Vocabulary, create_vocabulary

import warnings
# ignore some deprecation warnings
warnings.filterwarnings('ignore')

## Functions

In [12]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

def pad_sequence(tokenizer_array, desired_length):
    padded_sequence = pad_sequences([tokenizer_array], maxlen=desired_length, padding='post')[0]
    return padded_sequence

# Load main data set

In [5]:
df = pd.read_csv('retrosynthesis-all', header=None)
df['source'] = df[0].apply(lambda x: x.split('>>')[0])
df['target'] = df[0].apply(lambda x: x.split('>>')[1])
df.drop(0, axis=1, inplace=True)

In [7]:
tk = SMILESTokenizer()
vocabulary = Vocabulary()

#### Tokenization example

In [14]:
# smi_sample = df['source'].iloc[123]
smi_sample = 'CCBr'

# create a vocabulary using all SMILES in df
smiles_dataset = df['source'].unique().tolist() + df['target'].unique().tolist()
smiles_dataset = np.unique(smiles_dataset).tolist()
vocabulary = create_vocabulary(smiles_list=smiles_dataset, tokenizer=tk)
print(f'There are {len(vocabulary)} unique tokens in the vocabulary.\n')

tokenized_smi_sample = tk.tokenize(smi_sample, with_begin_and_end=False)
print(smi_sample)
print(tokenized_smi_sample)
print(vocabulary.encode(tokenized_smi_sample))

There are 86 unique tokens in the vocabulary.

CCBr
['C', 'C', 'Br']
[21. 21. 20.]


In [17]:
def preprocess_smiles_data(X):
    for row in range(len(X)):
        X[row] = tk.tokenize(X[row])
        X[row] = vocabulary.encode(X[row] )
        X[row]  = pad_sequence(X[row], 200 )
        X[row]  = torch.tensor([X[row] ])
    return X

x = [smi_sample]
preprocess_smiles_data(x)

[tensor([[ 1, 21, 21, 20,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0]], dtype=torch.int32)]

# RNN section

# Train / validation / test split

In [ ]:
from sklearn.model_selection import train_test_split

print(df.shape)

# Splitting the data into train and combined val/test sets
train_data, val_test_data = train_test_split(df, test_size=0.2, random_state=42)

# Splitting the combined val/test set into separate val and test sets
val_data, test_data = train_test_split(val_test_data, test_size=0.2, random_state=42)

# Printing the sizes of the resulting splits
print("Train data size:", len(train_data))
print("Validation data size:", len(val_data))
print("Test data size:", len(test_data))

In [ ]:
train     = train_data.copy()
test      = test_data.copy()
valid     = val_data.copy()

# create a vocabulary using all SMILES in df
dataset = df['source'].unique().tolist()+ df['target'].unique().tolist()
dataset = np.unique(dataset).tolist()

tokenizer = SMILESTokenizer()
vocab     = create_vocabulary(smiles_list=dataset,
                                    tokenizer=tokenizer,
                                    canonical=False)

MAX_LENGTH = max(len(v) for v in dataset)

print(f'There are {len(vocabulary)} unique tokens in the vocabulary.\n')
print(f'Max length: {MAX_LENGTH}.\n')
# print(f'The unique tokens are: \n{vocabulary.tokens()}')

In [ ]:
tk = SMILESTokenizer()
vocab = Vocabulary()

smi_sample = 'CCBr'
tokenized_smi_sample = tk.tokenize(smi_sample, with_begin_and_end=False)
print(tokenized_smi_sample)
vocabulary.encode(tokenized_smi_sample)

### Function for pad sequencing

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

def pad_sequence(tokenizer_array, desired_length):
    padded_sequence = pad_sequences([tokenizer_array], maxlen=desired_length, padding='post')[0]
    return padded_sequence

In [ ]:
for d in [train, test, valid]:
    for c in d.columns:
        d[c] = d[c].apply(lambda x: tk.tokenize(x, with_begin_and_end=False))
        d[c] = d[c].apply(lambda x: vocabulary.encode(x).astype(int))
        d[c] = d[c].apply(lambda x: pad_sequence(x, MAX_LENGTH))

# Model Building

In [ ]:
# Convert the source and target columns into numpy arrays
trainX = np.array(train['source'].tolist())
trainY = np.array(train['target'].tolist())

print(trainX.shape, trainY.shape)

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding

# Define the input shape
input_shape = (trainX.shape[1], 1)  # Assuming you want to feed one feature at a time

# Build the LSTM model
model = Sequential()
model.add(Embedding(input_dim=NUM_EMBEDDING, output_dim=EMBEDDING_DIM, input_length=MAX_LENGTH))
model.add(LSTM(units=128, input_shape=input_shape))
model.add(Dense(units=trainY.shape[1], activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(trainX, trainY, epochs=3, batch_size=2048)

In [ ]:
# Convert the source and target columns into numpy arrays
testX = np.array(test['source'].tolist())
testY = np.array(test['target'].tolist())

print(testX.shape, testY.shape)

In [ ]:
pred = model.predict(testX)
pd.DataFrame(pred)

In [ ]:
pd.DataFrame(testY)